# Lecture 31: Least Squares

In [ ]:
from datascience import *
import numpy as np

import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
%matplotlib inline

## Linear regression

In [ ]:
def standard_units(any_numbers):
    "Convert any array of numbers to standard units."
    return (any_numbers - np.mean(any_numbers)) / np.std(any_numbers)  

def correlation(t, x, y):
    """Return the correlation coefficient (r) of two variables."""
    return np.mean(standard_units(t.column(x)) * standard_units(t.column(y)))

def slope(t, x, y):
    """The slope of the regression line (original units)."""
    r = correlation(t, x, y)
    return r * np.std(t.column(y)) / np.std(t.column(x))

def intercept(t, x, y):
    """The intercept of the regression line (original units)."""
    return np.mean(t.column(y)) - slope(t, x, y) * np.mean(t.column(x))

In [ ]:
# From an actual class, but anonymized
exams = Table().read_table('exams.csv')
exams.show(3)

In [ ]:
# Does linear regression look appropriate?
exams.scatter('prelim', fit_line=True)
plots.xlim(35, 105);
plots.ylim(35, 105);

In [ ]:
r = correlation(exams, 'prelim', 'final')
s = slope(exams, 'prelim', 'final')
i = intercept(exams, 'prelim', 'final')
print('Correlation: ', r)
print('Slope:       ', s)
print('Intercept:   ', i)

**Q:** Which equation would you use to predict a final exam score $y$ given a prelim score $x$, both in original units (percent)?

A. $y = r \times x$  
B. $y = r \times x + i$  
C. $y = s \times x + i$  
D. none of the above


In [ ]:
def predict_final(prelim):
    return s * prelim + i

In [ ]:
predict_final(100) # regression to the mean?

In [ ]:
predict_final(30)

In [ ]:
exams.scatter('prelim')
plots.xlim(35, 105);
plots.ylim(35, 105);

# plot the regression line
x = make_array(40, 100)
y = s * x + i
plots.plot(x, y);

## Abuses of $r$

**1. Summarizing non-linear data**

In [ ]:
new_x = np.arange(-4, 4.1, 0.5)
nonlinear = Table().with_columns(
        'x', new_x,
        'y', new_x**2
    )
nonlinear.scatter('x', 'y', s=30, color='r')

In [ ]:
correlation(nonlinear, 'x', 'y')

There is definitely a relationship between $x$ and $y$.  It's just not linear.

**2. Eliminating outliers to "improve" $r$**

In [ ]:
line = Table().with_columns(
        'x', [1, 2, 3, 4],
        'y', [1, 2, 3, 4]
    )
line.scatter('x', 'y', s=30, color='r')

In [ ]:
correlation(line, 'x', 'y')

In [ ]:
outlier = Table().with_columns(
        'x', [1, 2, 3, 4, 5],
        'y', [1, 2, 3, 4, 0]
    )
outlier.scatter('x', 'y', s=30, color='r')

In [ ]:
correlation(outlier, 'x', 'y')

Because weight that outlier contributes to product is huge and offsets the linearity of the rest of the data.  **Do not rush to eliminate outliers from data.**  Maybe that data point is a measurement error, but maybe it indicates underlying population is non-linear.

**3. Drawing conclusions about individuals based on data about groups**

In [ ]:
# Participation rate = % of high school seniors who took SAT
# Scores are average scores across state
sat2014 = Table.read_table('sat2014.csv')
sat2014.sort('Combined', descending=True)

In [ ]:
sat2014.scatter('Participation Rate', 'Combined')
correlation(sat2014, 'Participation Rate', 'Combined')

**Q:** States with higher participation rates have lower average scores.  Why?

In [ ]:
sat2014.scatter('Critical Reading', 'Math')

In [ ]:
correlation(sat2014, 'Critical Reading', 'Math')

**Q:** Do students with high reading scores also have high math scores?

A. Yes  
B. No  
C. Maybe


## Quantifying error

In [ ]:
def read_file(f):
    with open (f, 'r') as file:
        data=file.read()
    return data
lw_text = read_file('little_women.txt')
lw_chapters = lw_text.split('CHAPTER ')[1:]
lw = Table().with_columns(
    'Periods', np.char.count(lw_chapters, '.'),
    'Characters', np.vectorize(len)(lw_chapters),
)

lw.show(3)

In [ ]:
lw.scatter('Periods', 'Characters')

In [ ]:
correlation(lw, 'Periods', 'Characters')

In [ ]:
def linear_predict(t, x, y):
    """Return the height of the regression line at each x value."""
    s = slope(t, x, y)
    i = intercept(t, x, y)
    return s * t.column(x) + i

In [ ]:
lw_predicted = lw.with_column(
    'Predicted', linear_predict(lw, 'Periods', 'Characters')
)
lw_predicted.scatter('Periods')

In [ ]:
sample = [[131, 14431], [231, 20558], [392, 40935], [157, 23524]]
def lw_errors(slope, intercept):
    print('Slope:    ', np.round(slope), 'characters per period')
    print('Intercept:', np.round(intercept), 'characters')
    lw.scatter('Periods', 'Characters')
    xlims = np.array([50, 450])
    plots.plot(xlims, slope * xlims + intercept, lw=2)
    for x, y in sample:
        plots.plot([x, x], [y, slope * x + intercept], color='r', lw=2)

In [ ]:
lw_reg_slope = slope(lw, 'Periods', 'Characters')
lw_reg_intercept = intercept(lw, 'Periods', 'Characters')
lw_errors(lw_reg_slope, lw_reg_intercept)

In [ ]:
error = lw_predicted.column('Characters') - lw_predicted.column('Predicted')
lw_predicted_error = lw_predicted.with_column(
    'Error', error
)
lw_predicted_error.show(3)

**Q:** What would be a good way to summarize all those errors as a single number?

## RMSE

In [ ]:
lw_predicted_error_sq = lw_predicted_error.with_column(
    'Squared Error', error ** 2
)
lw_predicted_error_sq.show(3)

In [ ]:
mean_squared_error = np.mean(lw_predicted_error_sq.column('Squared Error'))
mean_squared_error

In [ ]:
root_mean_squared_error = np.sqrt(mean_squared_error)
root_mean_squared_error

rmse = SD($y$) $\times \sqrt{1 - r^2}$

In [ ]:
np.sqrt(1 - correlation(lw, 'Periods', 'Characters')**2) * np.std(lw.column('Characters'))

## Comparing lines with RMSE

In [ ]:
def lw_visualize_rmse(slope, intercept):
    lw_errors(slope, intercept)
    x = lw.column('Periods')
    y = lw.column('Characters')
    predicted = slope * x + intercept
    mse = np.mean((y - predicted) ** 2)
    print("RMSE:     ", mse ** 0.5)

In [ ]:
lw_visualize_rmse(50, 10000)

**Q:** Which way should I adjust slope to reduce error?

A. Up  
B. Down  
C. I'm not sure  

In [ ]:
lw_reg_slope = slope(lw, 'Periods', 'Characters')
lw_reg_intercept = intercept(lw, 'Periods', 'Characters')
lw_visualize_rmse(lw_reg_slope, lw_reg_intercept)

## Finding the line with least RMSE

In [ ]:
def f(x):
    return (x - 3) ** 2 + 1

**Q:** What value of `x` will produce the smallest value of `f(x)`?

A. 0  
B. 1  
C. 2  
D. 3  
E. none of the above

In [ ]:
x = np.arange(0, 6.1, .1)
y = f(x)
Table().with_columns('x', x, 'y', y).scatter('x')

In [ ]:
minimize(f)

In [ ]:
f(minimize(f))

In [ ]:
def lw_rmse(any_slope, any_intercept):
    """Compute the RMSE for a line through the Little Women data.
    The line has the slope and intercept given as arguments."""
    x = lw.column('Periods')
    y = lw.column('Characters')
    predicted = any_slope*x + any_intercept
    return np.sqrt(np.mean((y - predicted) ** 2))

In [ ]:
lw_rmse(50, 10000)

In [ ]:
best = minimize(lw_rmse)
best

In [ ]:
lw_rmse(best.item(0), best.item(1))

In [ ]:
make_array(lw_reg_slope, lw_reg_intercept)

In [ ]:
lw_rmse(lw_reg_slope, lw_reg_intercept)